In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [13]:
import pandas as pd
import numpy as np

import os
import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from IPython.display import Audio

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization, SimpleRNN, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
from matplotlib import image
import matplotlib.pyplot as plt
import os
import numpy as np

In [3]:
os.chdir('/Users/zeyuanli/Dropbox/Github/Librosa test'+'/AudioWAV')
audioFiles = os.listdir()
i = 1
mfcc_loaded = []
for audio in audioFiles:
    if audio.endswith('.wav'):
      if i % 500 == 0:
        print(i)
      i+=1
      audioName = audio.split(".")[0]
      signal, sr = librosa.load(f"{audio}")
      mfcc_loaded.append(librosa.feature.melspectrogram(y=signal, sr=sr).T)

file_emotion = []
file_path = []

for file in audioFiles:
    # storing file paths
    if (file.endswith('.wav')):
        file_path.append(file)
        # storing file emotions
        part=file.split('_')
        if part[2] == 'SAD':
            file_emotion.append('sad')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Crema_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


,Emotions,Path
0,angry,1022_ITS_ANG_XX.wav
1,angry,1037_ITS_ANG_XX.wav
2,neutral,1060_ITS_NEU_XX.wav
3,neutral,1075_ITS_NEU_XX.wav
4,disgust,1073_IOM_DIS_XX.wav


In [5]:
X = tf.keras.preprocessing.sequence.pad_sequences(mfcc_loaded)
X.shape
Y = np.array(data_path.Emotions)
encoder = OneHotEncoder()
Y = encoder.fit_transform(Y.reshape(-1,1)).toarray()

# making our data compatible to model.
#X = np.expand_dims(X, axis=-1)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(Y), test_size=0.3)

In [7]:
print((X_train.shape))
print (y_train.shape)

(5209, 216, 128)
(5209, 6)


In [8]:
def LSTM_Model():
    
    model = tf.keras.Sequential()
    #model.add(SimpleRNN(128, input_shape=images_loaded[0], return_sequences=True))
    #model.add(Conv2D(256, kernel_size = (5,5), strides = 1,  padding = 'same', activation = 'relu', input_shape = (216,20,1)))
    model.add(LSTM(128, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(128))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(6, activation='softmax'))

    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    model.summary()
    return model


In [9]:
model = LSTM_Model()

Metal device set to: Apple M1 Pro


2021-12-07 16:11:22.766489: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 16:11:22.766885: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 216, 128)          131584    
_________________________________________________________________
dropout (Dropout)            (None, 216, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 271,814
Trainable params: 271,814
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(X_train, y_train, validation_split = 0.1, batch_size=32, epochs=50)

Epoch 1/50
147/147 [==============================] - 21s 142ms/step - loss: 1.3973 - accuracy: 0.4241 - val_loss: 1.4859 - val_accuracy: 0.4012
Epoch 2/50
147/147 [==============================] - 21s 141ms/step - loss: 1.3820 - accuracy: 0.4366 - val_loss: 1.5131 - val_accuracy: 0.3781
Epoch 3/50
147/147 [==============================] - 21s 141ms/step - loss: 1.3838 - accuracy: 0.4311 - val_loss: 1.4842 - val_accuracy: 0.4107
Epoch 4/50
147/147 [==============================] - 21s 141ms/step - loss: 1.3852 - accuracy: 0.4279 - val_loss: 1.5108 - val_accuracy: 0.3800
Epoch 5/50
147/147 [==============================] - 21s 141ms/step - loss: 1.3628 - accuracy: 0.4420 - val_loss: 1.5675 - val_accuracy: 0.3762
Epoch 6/50
147/147 [==============================] - 21s 141ms/step - loss: 1.4629 - accuracy: 0.3895 - val_loss: 1.5288 - val_accuracy: 0.3992
Epoch 7/50
147/147 [==============================] - 21s 143ms/step - loss: 1.3954 - accuracy: 0.4251 - val_loss: 1.4861 - val_ac

In [16]:
pred = model.predict(X_train)
pred_test = model.predict(X_test)

print("accuracy score for Train set is:", accuracy_score(np.argmax(y_train,axis =1), np.argmax(pred,axis=1)))
print("accuracy score for Test set is: ", accuracy_score(np.argmax(y_test,axis =1), np.argmax(pred_test,axis=1)))
print("[Anger, disgust, fear, happy, neutral, sad]")
print("F1 score for Train set is:", f1_score(np.argmax(y_train,axis =1), np.argmax(pred,axis=1), average = None))
print("F1 score for Test set is: ", f1_score(np.argmax(y_test,axis =1), np.argmax(pred_test,axis=1), average = None))

accuracy score for Train set is: 0.5231330389710117
accuracy score for Test set is:  0.3994626063591581
[Anger, disgust, fear, happy, neutral, sad]
F1 score for Train set is: [0.71043538 0.31382528 0.50608788 0.55398633 0.47512577 0.53673788]
F1 score for Test set is:  [0.45390071 0.21218075 0.35939323 0.4244898  0.41324503 0.47940075]
